In [1]:
import transformers
transformers.__version__

'4.28.1'

In [ ]:
import torch

In [9]:
%pip install datasets

In [39]:
!pip install bitsandbytes

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

In [ ]:

datasets["test"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [12]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [13]:

show_random_elements(datasets["train"])

,text
0,"Made on a budget of around ₹ 100 — 120 million , the film 's principal photography commenced in November 2005 and lasted until April 2006 . Most of the film was shot in and around Hyderabad and Chennai , except for a song which was shot at the province of Phuket in Thailand and the city of Bangkok . Shyam K. Naidu was the film 's cinematographer , and it was edited by Marthand K. Venkatesh . The soundtrack and background score were composed by Mani Sharma . \n"
1,
2,
3,"It is a sign of Edward 's high regard for Lancaster that he would bestow such extensive privileges on him . The two men were second cousins through their great @-@ grandfather Henry III and practically coeval ( Edward was born in 1312 ) , so it is natural to assume that a strong sense of camaraderie existed between them . Another factor that might have influenced the king 's decision was the fact that Henry had no male heir , so the grant was made for the Earl 's lifetime only , and not intended to be hereditary . \n"
4,
5,"After many delays , the General and his party arrive in Honda , where the Governor , Posada Gutiérrez , has arranged for three days of fiestas . On his last night in Honda , the General returns late to camp and finds one of his old friends , Miranda Lyndsay , waiting for him . The General recalls that fifteen years ago , she had learned of a plot against his life and had saved him . The following morning , the General begins the voyage down the Magdalena River . Both his physical debilitation and pride are evident as he negotiates the slope to the dock : he is in need of a sedan chair but refuses to use it . The group stays a night in Puerto Real , where the General claims he sees a woman singing during the night . His aides @-@ de @-@ camp and the watchman conduct a search , but they fail to uncover any sign of a woman having been in the vicinity . \n"
6,"Many Egyptologists and anthropologists have suggested theories about how the gods developed in these early times . Gustave Jéquier , for instance , thought the Egyptians first revered primitive fetishes , then deities in animal form , and finally deities in human form , whereas Henri Frankfort argued that the gods must have been envisioned in human form from the beginning . Some of these theories are now regarded as too simplistic , and more current ones , such as Siegfried Morenz ' hypothesis that deities emerged as humans began to distinguish themselves from and personify their environment , are difficult to prove . \n"
7,"Disassembly of the Type 94 Nambu pistol is considered difficult and can lead to damage to the pistol if done carelessly . After clearing the Type 94 , the operator must draw the slide against the magazine follower to hold the bolt to the rear of the pistol . This will allow the crossbolt to be released after the firing pin is depressed . Removal of the crossbolt without depressing the firing pin will damage both the firing pin and the crossbolt . Removing the crossbolt is further complicated as the disassemblers hands are both holding the pistol and depressing the firing pin . \n"
8,
9,"Suresh Krishnamoorthy of The Hindu summarised , "" Magadheera is not for the weak @-@ hearted , those who do not like the sight of blood and neither is it for those who like movies with storylines that are much @-@ closer @-@ to @-@ everyday- reality . But Rajamouli excels in story @-@ telling . The way he has used the flashback as a flip switch , going back and forth and taking the viewer through a 400 @-@ year journey in a jiffy is interesting . The rest is about how well technology , creativity , imagination and innovation are leveraged to present what is an eye @-@ pleasing experience for viewers . "" B. V. S. Prakash of The Times of India wrote , "" Despite a few narrative lapses , the much @-@ hyped semi @-@ periodic epic lives up to expectations . Unlike his previous action @-@ centric films , director Rajamouli dishes a heart @-@ touching love story in a lavish canvas c

In [14]:
model_checkpoint = "distilgpt2"

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,)

In [18]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [20]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-262948adfd561d96_*_of_00004.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-f2fa497fbe27fc6c_*_of_00004.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-4d919120a52d1192_*_of_00004.arrow


https://openai.com/research/better-language-models

In [33]:
datasets["train"][5]

{'text': " It met with positive sales in Japan , and was praised by both Japanese and western critics . After release , it received downloadable content , along with an expanded edition in November of that year . It was also adapted into manga and an original video animation series . Due to low sales of Valkyria Chronicles II , Valkyria Chronicles III was not localized , but a fan translation compatible with the game 's expanded edition was released in 2014 . Media.Vision would return to the franchise with the development of Valkyria : Azure Revolution for the PlayStation 4 . \n"}

In [21]:
tokenized_datasets["train"][1]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
# block_size = tokenizer.model_max_length
block_size = 128

In [23]:
def group_texts(examples):
    # Concatenate all texts.
    # Create a new dictionary by iterating over the keys in the input `examples`
    # dictionary and concatenating the lists of tokens for each key.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    
    # Calculate the total length of the concatenated list of tokens
    # for the first key in the `examples` dictionary.
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    
    # Drop the small remainder that is smaller than `block_size`.
    # You could modify this part to pad the text if the model supports padding
    # instead of dropping the remainder.
    total_length = (total_length // block_size) * block_size
    
    # Split the concatenated lists of tokens into chunks of size `block_size`.
    # Create a new dictionary by iterating over the `concatenated_examples`
    # dictionary and using list comprehension to create a list of chunks
    # for each key in the dictionary.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    
    # Add the "labels" key to the `result` dictionary with the same value
    # as the "input_ids" key.
    result["labels"] = result["input_ids"].copy()
    
    return result


In [25]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-5fa659d1b8d5ed60_*_of_00004.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-502f4991e270aa62_*_of_00004.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-eb096a1839e0dbce_*_of_00004.arrow


In [37]:
#tokenizer.decode(lm_datasets["train"][1]["input_ids"])
#lm_datasets["train"][1]["input_ids"]

[983,
 290,
 5679,
 262,
 366,
 17871,
 5321,
 366,
 837,
 257,
 23634,
 2422,
 4326,
 7351,
 262,
 3277,
 286,
 7096,
 544,
 1141,
 262,
 5498,
 1898,
 6839,
 1810,
 508,
 1620,
 3200,
 2042,
 4560,
 290,
 389,
 46852,
 1028,
 262,
 11773,
 4326,
 366,
 2199,
 321,
 265,
 88,
 12552,
 366,
 764,
 220,
 198,
 383,
 983,
 2540,
 2478,
 287,
 3050,
 837,
 6872,
 625,
 257,
 1588,
 6903,
 286,
 262,
 670,
 1760,
 319,
 569,
 18354,
 7496,
 17740,
 2873,
 764,
 2893,
 340,
 17383,
 262,
 3210,
 3033,
 286,
 262,
 2168,
 837,
 340,
 635,
 25289,
 3294,
 16895,
 837,
 884,
 355,
 1642,
 262,
 983,
 517,
 43486,
 329,
 2168,
 29661,
 764,
 15684,
 11915,
 371,
 4548,
 64,
 8835,
 73,
 280,
 290,
 26777,
 7286,
 13704,
 13231,
 43354,
 1111,
 4504,
 422,
 2180,
 12784,
 837,
 1863,
 351,
 569,
 18354,
 7496,
 17740,
 2873,
 3437,
 33687,
 5303,
 18024]

In [40]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [41]:
from transformers import Trainer, TrainingArguments

In [44]:
nvidia-smi

UsageError: Line magic function `%nvidia-smi` not found.


In [45]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [48]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=7002, training_loss=3.6940719264808703, metrics={'train_runtime': 608.2752, 'train_samples_per_second': 92.06, 'train_steps_per_second': 11.511, 'total_flos': 1829011929956352.0, 'train_loss': 3.6940719264808703, 'epoch': 3.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [54]:
math.exp(eval_results['eval_loss'])

38.172572558779414

In [56]:
model.save_pretrained("/mnt/CEPH/transformers/distillgpt2/")
tokenizer.save_pretrained("/mnt/CEPH/transformers/distillgpt2/")


('/mnt/CEPH/transformers/distillgpt2/tokenizer_config.json',
 '/mnt/CEPH/transformers/distillgpt2/special_tokens_map.json',
 '/mnt/CEPH/transformers/distillgpt2/vocab.json',
 '/mnt/CEPH/transformers/distillgpt2/merges.txt',
 '/mnt/CEPH/transformers/distillgpt2/added_tokens.json',
 '/mnt/CEPH/transformers/distillgpt2/tokenizer.json')

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained("path/to/your/local/model/directory")
tokenizer = AutoTokenizer.from_pretrained("path/to/your/local/model/directory")


In [67]:
from transformers import pipeline

generator = pipeline("text-generation", model="/mnt/CEPH/transformers/distillgpt2/")

In [68]:
prompt = "Somatic hypermutation allows the immune system to"
generator(prompt)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Somatic hypermutation allows the immune system to attack any species in an attempt to damage them by making it more like a " virus " or " virus being distributed over the body. \n = = Diagnosis and diagnosis = = \n When'}]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [63]:
!pip install gradio

In [66]:

import gradio as gr

demo = gr.Interface.from_pipeline(pipe)
demo.launch(share=True)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 83, but `max_length` is set to 50. This can lead to unexpected behavior.